<a href="https://colab.research.google.com/github/andreidore/machine_learning/blob/main/bert/classification_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://towardsdatascience.com/multi-label-multi-class-text-classification-with-bert-transformer-and-keras-c6355eccb63a

In [1]:
%pip install transformers

     |████████████████████████████████| 1.5MB 14.4MB/s 
     |████████████████████████████████| 890kB 59.0MB/s 
     |████████████████████████████████| 2.9MB 63.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=eb08155425e5236c12cb6664e0c71aaf6307b7458e43a40d7b8fc22a7f067616
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import tensorflow as tf
from transformers import TFBertModel,  BertConfig, BertTokenizerFast

from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

import pandas as pd
from sklearn.model_selection import train_test_split


In [3]:
print(tf.__version__)

2.4.0


In [4]:
!wget http://files.consumerfinance.gov/ccdb/complaints.csv.zip

--2021-01-01 17:46:45--  http://files.consumerfinance.gov/ccdb/complaints.csv.zip
Resolving files.consumerfinance.gov (files.consumerfinance.gov)... 13.32.240.115, 13.32.240.85, 13.32.240.91, ...
Connecting to files.consumerfinance.gov (files.consumerfinance.gov)|13.32.240.115|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.consumerfinance.gov/ccdb/complaints.csv.zip [following]
--2021-01-01 17:46:45--  https://files.consumerfinance.gov/ccdb/complaints.csv.zip
Connecting to files.consumerfinance.gov (files.consumerfinance.gov)|13.32.240.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 302309786 (288M) [binary/octet-stream]
Saving to: ‘complaints.csv.zip’

complaints.csv.zip  100%[===================>] 288.30M   279MB/s    in 1.0s    

2021-01-01 17:46:46 (279 MB/s) - ‘complaints.csv.zip’ saved [302309786/302309786]



In [5]:
!unzip complaints.csv.zip

Archive:  complaints.csv.zip
  inflating: complaints.csv          


In [6]:
data = pd.read_csv('complaints.csv')

In [7]:
data = data[['Consumer complaint narrative', 'Product', 'Issue']]
data = data.dropna()
data = data.groupby('Issue').filter(lambda x : len(x) > 1)
data = data.groupby('Product').filter(lambda x : len(x) > 1)

# Set your model output as categorical and save in new label col
data['Issue_label'] = pd.Categorical(data['Issue'])
data['Product_label'] = pd.Categorical(data['Product'])
# Transform your output to numeric
data['Issue'] = data['Issue_label'].cat.codes
data['Product'] = data['Product_label'].cat.codes


In [8]:
data.head()

,Consumer complaint narrative,Product,Issue,Issue_label,Product_label
1,transworld systems inc. \nis trying to collect...,7,13,Attempts to collect debt not owed,Debt collection
3,"Over the past 2 weeks, I have been receiving e...",7,33,Communication tactics,Debt collection
7,Pioneer has committed several federal violatio...,7,13,Attempts to collect debt not owed,Debt collection
9,"Previously, on XX/XX/XXXX, XX/XX/XXXX, and XX/...",6,110,Problem with a credit reporting company's inve...,"Credit reporting, credit repair services, or o..."
10,Hello This complaint is against the three cred...,6,110,Problem with a credit reporting company's inve...,"Credit reporting, credit repair services, or o..."


In [9]:
data.shape

(647424, 5)

In [10]:
data, data_test = train_test_split(data, test_size = 0.2, stratify = data[['Issue']])

In [11]:
data.shape

(517939, 5)

In [12]:
# Name of the BERT model to use
model_name = 'bert-base-uncased'

# Max length of tokens
max_length = 100

# Load transformers config and set output_hidden_states to False
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False

# Load BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)

# Load the Transformers BERT model
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [13]:
bert=transformer_model.layers[0]

input_ids=Input(shape=(max_length,),name="input_ids",dtype="int32")
inputs={"input_ids":input_ids}

bert_model=bert(input_ids)[1]

dropout=Dropout(config.hidden_dropout_prob,name="pooled")
pooled_dropout=dropout(bert_model,training=False)
issue=Dense(units=len(data.Issue_label.value_counts()),kernel_initializer=TruncatedNormal(stddev=config.initializer_range),name="issue")(pooled_dropout)
product=Dense(units=len(data.Product_label.value_counts()),kernel_initializer=TruncatedNormal(stddev=config.initializer_range),name="product")(pooled_dropout)
outputs={"issue":issue,"product":product}

model=Model(inputs=inputs,outputs=outputs)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 109482240   input_ids[0][0]                  
__________________________________________________________________________________________________
pooled (Dropout)                (None, 768)          0           bert[0][1]                       
__________________________________________________________________________________________________
issue (Dense)                   (None, 159)          122271      pooled[0][0]                     
______________________________________________________________________________________________

In [ ]:
#######################################
### ------- Train the model ------- ###
# Set an optimizer
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss = {'issue': CategoricalCrossentropy(from_logits = True), 'product': CategoricalCrossentropy(from_logits = True)}
metric = {'issue': CategoricalAccuracy('accuracy'), 'product': CategoricalAccuracy('accuracy')}
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)
# Ready output data for the model
y_issue = to_categorical(data['Issue'])
y_product = to_categorical(data['Product'])
# Tokenize the input (takes some time)
x = tokenizer(
    text=data['Consumer complaint narrative'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)
# Fit the model
history = model.fit(
    x={'input_ids': x['input_ids']},
    y={'issue': y_issue, 'product': y_product},
    validation_split=0.2,
    batch_size=64,
    epochs=10)

Epoch 1/10
6475/6475 [==============================] - 2964s 455ms/step - loss: 3.6848 - issue_loss: 2.6565 - product_loss: 1.0282 - issue_accuracy: 0.3290 - product_accuracy: 0.6880 - val_loss: 3.0246 - val_issue_loss: 2.2271 - val_product_loss: 0.7975 - val_issue_accuracy: 0.3820 - val_product_accuracy: 0.7396
Epoch 2/10
6475/6475 [==============================] - 2945s 455ms/step - loss: 3.0157 - issue_loss: 2.2218 - product_loss: 0.7940 - issue_accuracy: 0.3825 - product_accuracy: 0.7423 - val_loss: 2.9386 - val_issue_loss: 2.1659 - val_product_loss: 0.7727 - val_issue_accuracy: 0.3977 - val_product_accuracy: 0.7437
Epoch 3/10
3009/6475 [============>.................] - ETA: 24:09 - loss: 2.9340 - issue_loss: 2.1680 - product_loss: 0.7660 - issue_accuracy: 0.3954 - product_accuracy: 0.7478

In [ ]:
#######################################
### ----- Evaluate the model ------ ###
# Ready test data
test_y_issue = to_categorical(data_test['Issue'])
test_y_product = to_categorical(data_test['Product'])
test_x = tokenizer(
    text=data_test['Consumer complaint narrative'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)
# Run evaluation
model_eval = model.evaluate(
    x={'input_ids': test_x['input_ids']},
    y={'issue': test_y_issue, 'product': test_y_product}
)